In [9]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input
from PIL import Image

In [10]:
import pandas as pd
from IPython.display import display

# Đọc dữ liệu
data_train = pd.read_csv('emotion_dataset.csv')
data_val = pd.read_csv('emotion_dataset_validation.csv')

# Hiển thị song song 2 bảng đầu tiên
print("📘 Training Dataset:")
display(data_train.head(20))

print("📗 Validation Dataset:")
display(data_val.head(20))

📘 Training Dataset:


,emotion,pixels,Usage
0,0,72 78 81 75 59 54 63 61 55 55 62 46 93 114 121...,Training
1,0,152 149 147 157 146 133 114 138 170 175 184 15...,Training
2,0,29 25 21 23 26 24 49 67 85 101 121 125 130 140...,Training
3,0,32 23 20 56 43 34 38 46 92 99 34 21 27 27 31 4...,Training
4,0,222 218 202 189 199 208 193 134 103 89 37 41 5...,Training
5,0,238 239 237 234 232 228 231 203 170 152 166 18...,Training
6,0,35 26 25 14 12 14 17 20 22 22 24 24 46 35 29 2...,Training
7,0,120 119 120 121 121 122 122 123 122 120 124 12...,Training
8,0,255 255 254 255 250 223 179 125 119 106 96 97 ...,Training
9,0,73 77 79 105 130 128 92 81 76 55 58 44 71 76 7...,Training


📗 Validation Dataset:


,emotion,pixels,Usage
0,0,58 66 70 77 117 154 137 108 76 70 76 82 88 80 ...,Validation
1,0,23 26 21 9 6 19 33 11 3 63 89 73 39 33 42 36 2...,Validation
2,0,201 182 182 184 205 204 203 220 223 228 231 23...,Validation
3,0,93 86 78 78 80 92 109 99 104 107 114 130 148 1...,Validation
4,0,11 6 1 0 0 1 0 0 2 0 0 1 0 2 3 0 0 0 1 0 0 0 1...,Validation
5,0,93 93 91 92 90 94 78 75 147 104 81 117 102 100...,Validation
6,0,5 11 9 11 14 13 15 1 26 34 35 19 40 73 39 64 7...,Validation
7,0,41 4 1 1 26 67 94 122 113 107 117 131 136 127 ...,Validation
8,0,161 150 156 177 198 205 205 210 211 211 207 19...,Validation
9,0,67 75 81 83 90 104 113 111 127 134 144 144 148...,Validation


In [11]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
# ------------------ XỬ LÝ TẬP TRAIN ------------------
x_train_data = data_train['pixels']
y_train_data = data_train['emotion']

# Giải quyết mất cân bằng
oversampler = RandomOverSampler()
x_train_data, y_train_data = oversampler.fit_resample(x_train_data.values.reshape(-1,1), y_train_data)

# Chuyển đổi pixel thành mảng float32
x_train_data = pd.Series(x_train_data.flatten())
x_train = np.array(list(map(str.split, x_train_data)), dtype=np.float32) / 255.0
x_train = x_train.reshape(-1, 48, 48, 1)

# One-hot encoding nhãn
y_train = to_categorical(y_train_data, num_classes=7)

# ------------------ XỬ LÝ TẬP VALIDATION ------------------
x_val_data = data_val['pixels']
y_val_data = data_val['emotion']

x_val = np.array(list(map(str.split, x_val_data)), dtype=np.float32) / 255.0
x_val = x_val.reshape(-1, 48, 48, 1)
y_val = to_categorical(y_val_data, num_classes=7)

# ------------------ IN RA KÍCH THƯỚC ------------------
print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_val  :", x_val.shape)
print("y_val  :", y_val.shape)

x_train: (50148, 48, 48, 1)
y_train: (50148, 7)
x_val  : (7066, 48, 48, 1)
y_val  : (7066, 7)


In [12]:

from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

# 2️⃣ ------------------ TẠO MÔ HÌNH CNN ------------------
model = Sequential([
    Input((48, 48, 1)),
    Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    Dropout(0.25),

    Conv2D(64, (3,3), strides=(1,1), padding='same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),

    Conv2D(128, (3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    Dropout(0.25),

    Conv2D(128, (3,3), strides=(1,1), padding='same'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),

    Conv2D(256, (3,3), strides=(1,1), padding='valid'),
    BatchNormalization(axis=3),
    Activation('relu'),
    MaxPooling2D((2,2)),

    Flatten(),

    Dense(128, activation='relu'),
    Dropout(0.25),

    Dense(256, activation='relu'),
    Dropout(0.2),

    Dense(7, activation='softmax')
])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 46, 46, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 46, 46, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_10 (Activation)      │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 46, 46, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 46, 46, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 46, 46, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 21, 21, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 21, 21, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 21, 21, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 21, 21, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 21, 21, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 21, 21, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 8, 8, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_14 (Activation)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,115,975 (4.26 MB)

 Trainable params: 1,114,695 (4.25 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [14]:
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
# 5️⃣ ------------------ COMPILE & TRAIN ------------------
adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val))

# 6️⃣ ------------------ ĐÁNH GIÁ & LƯU MODEL ------------------
loss, acc = model.evaluate(x_val, y_val)
print(f"✅ Accuracy on validation: {acc*100:.2f}%")
print(f"✅ Loss on validation: {loss:.4f}")

model.save('emotion_model_version3.keras')
print("✅ Model đã lưu thành công vào 'emotion_model_version3.keras'")

Epoch 1/50
1568/1568 ━━━━━━━━━━━━━━━━━━━━ 362s 229ms/step - accuracy: 0.2404 - loss: 1.8705 - val_accuracy: 0.4135 - val_loss: 1.5448
Epoch 2/50
1568/1568 ━━━━━━━━━━━━━━━━━━━━ 360s 229ms/step - accuracy: 0.4372 - loss: 1.4625 - val_accuracy: 0.4939 - val_loss: 1.3323
Epoch 3/50
1568/1568 ━━━━━━━━━━━━━━━━━━━━ 360s 229ms/step - accuracy: 0.5203 - loss: 1.2565 - val_accuracy: 0.5207 - val_loss: 1.2732
Epoch 4/50
1568/1568 ━━━━━━━━━━━━━━━━━━━━ 360s 230ms/step - accuracy: 0.5711 - loss: 1.1274 - val_accuracy: 0.5292 - val_loss: 1.2344
Epoch 5/50
1568/1568 ━━━━━━━━━━━━━━━━━━━━ 360s 230ms/step - accuracy: 0.6006 - loss: 1.0409 - val_accuracy: 0.5590 - val_loss: 1.1841
Epoch 6/50
1568/1568 ━━━━━━━━━━━━━━━━━━━━ 361s 230ms/step - accuracy: 0.6283 - loss: 0.9664 - val_accuracy: 0.5760 - val_loss: 1.1498
Epoch 7/50
1568/1568 ━━━━━━━━━━━━━━━━━━━━ 361s 230ms/step - accuracy: 0.6548 - loss: 0.9088 - val_accuracy: 0.5737 - val_loss: 1.1540
Epoch 8/50
1568/1568 ━━━━━━━━━━━━━━━━━━━━ 361s 230ms/step - ac